In [1]:
import sys, os
from dotenv import load_dotenv

from llama_index.core import Settings
from llama_index.core.tools import FunctionTool
from llama_index.llms.groq import Groq
from llama_index.core.agent import ReActAgent

from llama_index.core.callbacks import CallbackManager
from langfuse.llama_index import LlamaIndexCallbackHandler

load_dotenv('../.env')

jinaai_api_key = os.environ.get("JINAAI_API_KEY")
groq_api_key = os.environ.get("GROQ_API_KEY")
lgfuse_pub_key = os.environ.get("LANGFUSE_PUBLIC_KEY")
lgfuse_secret_key = os.environ.get("LANGFUSE_SECRET_KEY")


langfuse_callback_handler = LlamaIndexCallbackHandler(
    public_key=lgfuse_pub_key,
    secret_key=lgfuse_secret_key,
    host="http://127.0.0.1:3000"
)

Settings.callback_manager = CallbackManager([langfuse_callback_handler])
Settings.llm = Groq(model="llama3-70b-8192", api_key=groq_api_key)

/home/aizat/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b

multiply_tool = FunctionTool.from_defaults(fn=multiply)
agent = ReActAgent.from_tools([multiply_tool], verbose=True)

In [4]:
agent.chat("What is 2123 * 21519993")

> Running step 406273dd-e6d3-472d-b4d3-d6d095bb0fbf. Step input: What is 2123 * 21519993
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2123, 'b': 21519993}
Observation: 45686945139
> Running step e66afa14-03bd-4e64-aae3-5fa235ffbf67. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: 45686945139


AgentChatResponse(response='45686945139', sources=[ToolOutput(content='45686945139', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2123, 'b': 21519993}}, raw_output=45686945139, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [5]:
agent.chat("What is 2123 - 21519993")

> Running step bc897706-a1db-4778-a04a-c261096bf624. Step input: What is 2123 - 21519993
Thought: The current language of the user is: English. I need to calculate the difference between two numbers.

I don't have a tool to directly calculate the difference, but I can use the multiply tool to calculate the product and then subtract.
Action: multiply
Action Input: {'a': -21519993, 'b': 1}
Observation: -21519993
> Running step 70e2324d-79b3-41ad-b9e8-2413e9b38424. Step input: None
Thought: Now I have the negative value of the second number, I can add it to the first number to get the difference.
Action: multiply
Action Input: {'a': 2123, 'b': 1}
Observation: 2123
> Running step b0faefe3-7add-4b50-b6cc-d420f9ac52d7. Step input: None
Thought: Now I have both numbers, I can add the negative value to the first number to get the difference.
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: -21297770


AgentChatResponse(response='-21297770', sources=[ToolOutput(content='-21519993', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': -21519993, 'b': 1}}, raw_output=-21519993, is_error=False), ToolOutput(content='2123', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2123, 'b': 1}}, raw_output=2123, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [ ]:
from llama_index.core.agent import AgentRunner

agent = AgentRunner.from_llm([multiply_tool], llm=llm, verbose=True)